In [0]:
pip install folium

In [0]:
pip install geopy

In [0]:
import folium
import pandas as pd
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim

In [0]:
def city(n):
    geolocator = Nominatim(user_agent="Planet Health")
    location = geolocator.geocode(n)
    
    if location is None:
        return ["", ""]

    lt = location.latitude if location.latitude is not None else '' if location is not None else ''
    lo = location.longitude if location.longitude is not None else '' if location is not None else ''
    return [lt, lo]

In [0]:
df_voos = pd.read_csv("VRA_102019.csv", delimiter=";", usecols=['sg_icao_origem', 'sg_icao_destino', 'dt_partida_real'])
df_aeroportos = pd.read_csv("aeroportos.csv", delimiter=";")
df_voos = df_voos.rename(columns={"sg_icao_origem":"sg_origem", "sg_icao_destino":"sg_destino", "dt_partida_real":"data"})

In [0]:
df_origen = df_aeroportos.rename(columns={"Sigla OACI":"sg_origem", "Descrição":"desc", "Cidade":"cidade_origem", "UF":"uf", "País":"pais_origem", "Continente":"continente"})
result = df_voos.merge(df_origen, on='sg_origem')
result = result[['sg_origem', 'cidade_origem', 'pais_origem', 'sg_destino', 'data']]

In [0]:
df_destino = df_aeroportos.rename(columns={"Sigla OACI":"sg_destino", "Descrição":"desc", "Cidade":"cidade_destino", "UF":"uf", "País":"pais_destino", "Continente":"continente"})
df_destino.head()
result = result.merge(df_destino, on='sg_destino')

In [0]:
result = result[['sg_origem', 'cidade_origem', 'pais_origem', 'sg_destino', 'cidade_destino', 'pais_destino', 'data']]
result['data'] = result['data'].str[:10]

In [0]:
voos_dia = pd.DataFrame({'voos' : result.groupby(["cidade_origem", "cidade_destino", "data"]).size()}).reset_index()
voos_dia = voos_dia.sort_values(by=['data']).reset_index()


In [387]:
geolocator = Nominatim(user_agent="Planet Health")
vo = pd.DataFrame(data=voos_dia['cidade_destino'].unique())
ivo = vo[0].map(lambda x: city(x))
ivo

0       [13.8000382, -88.9140683]
1       [-23.4430602, -46.524459]
2        [40.4167047, -3.7035825]
3          [-22.90556, -47.06083]
4       [-2.6344567, -56.7319324]
                  ...            
207     [-2.9147515, -41.7661953]
208     [-21.8377536, -48.492853]
209    [-20.5531437, -48.5697509]
210        [46.603354, 1.8883335]
211      [45.7933982, 18.2633066]
Name: 0, Length: 212, dtype: object

In [0]:
mapa = folium.Map(location=[location.latitude, location.longitude])
# mapa = folium.Map([48., 5.], tiles='stamentoner', zoom_start=6)
# HeatMap(r).add_to(mapa)
mapa